In [2]:
import os
import csv
from openai import OpenAI
from config import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [10]:
articles_csv_path = './articulos.csv'
articles = []
with open(articles_csv_path, 'r', newline='') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for row in csv_reader:
        article = {
            "title": row['title'],
            "date": row['date'],
            "content": row['content']
        }
        articles.append(article)

articles_subset = articles[0:3]
print(len(articles_subset))
print(articles_subset)

3
[{'title': 'Tesla: Nuevo León anuncia licitación para la construcción de un puente de acceso a la planta', 'date': '2023-11-09 21:51:10+00:00', 'content': 'El Gobierno de Nuevo León lanzó la licitación para la construcción de un puente de acceso a la planta de vehículos eléctricos de Tesla, dirigida por Elon Musk, en el municipio de Santa Catarina.Mediante un comunicado, el gobierno a cargo de Samuel García, informó que la convocatoria forma parte del Proyecto Integral de las obras necesarias para los accesos a la planta armadora de autos eléctricos.Las licitaciones consisten en un Puente Superior Vehicular, rampas de acceso y la ampliación de un tercer carril sobre la carretera libre Monterrey – Saltillo (…) Y son complemento a los estudios y levantamientos realizados en fechas anterioresexplica el comunicado.Así nos estamos preparando para la llegada de Tesla, con una movilidad de PRIMER MUNDO. Un nuevo puente, la ampliación de tres carriles de la carretera y nuevas rampas de acces

In [18]:
client = OpenAI()

def rank_article(article):
    try:
        # gpt_role = """Your role is to act as an Article Ranking Assistant. You will be provided with financial articles, each containing a title, date, and content. Your task is to analyze and rank these articles based on the following criteria:
        #     Scale: How many people the event affected? Rate this on a scale from 1 to 10.
        #     Magnitude: How big was the effect, potential: how likely it is that the event will cause bigger events? Rate this on a scale from 1 to 10.
        #     Immediacy: How close in time is the event? Rate this on a scale from 1 to 10.
        #     # For each article, provide a brief summary of your analysis and a total score out of 30, derived from the individual ratings for each criterion. Your evaluation will help in prioritizing these articles for readers who seek the most relevant and insightful financial news."""
        gpt_role = """Your role is to act as an Article Ranking Assistant. You will be provided with financial articles, each containing a title, date, and content. Your task is to analyze and rank these articles based on the following criteria:
            Scale: How many people the event affected? Rate this on a scale from 1 to 10.
            Magnitude: How big was the effect, potential: how likely it is that the event will cause bigger events? Rate this on a scale from 1 to 10.
            Immediacy: How close in time is the event? Rate this on a scale from 1 to 10.
            For each article, only provide a total score out of 30 nothing more, derived from the individual ratings for each criterion. Your evaluation will help in prioritizing these articles for readers who seek the most relevant and insightful financial news."""
        gpt_prompt = f"""Analyze this article and give me a total score out of 30 nothing more:
                            Title: {article['title']}
                            Date: {article['date']}
                            Content: {article['content']}"""
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": gpt_role},
                {"role": "user", "content": gpt_prompt}
            ]
        )
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [48]:
client = OpenAI()

def rank_article(article):
    try:
        gpt_role = """Your role is to act as an Article Ranking Assistant. You will be provided with financial articles, each containing title, date, and content. Your task is to rate these articles based on the following criteria (each on a scale from 1 to 10):
            Scale: How many people the event affected?
            Magnitude: How big was the effect?
            Potential: How likely it is that the event will cause bigger events?
            Immediacy: How close in time is the event?
            Significance: How much the event affects the financial market as a whole? 
            For each article, only provide a total score out of 50 nothing more in JSON format, derived from the individual ratings for each criterion. Your evaluation will help in prioritizing these articles for readers who seek the most relevant and insightful financial news."""
        gpt_prompt = f"""Analyze this article and give me a total score out of 30 nothing more:
                            Title: {article['title']}
                            Date: {article['date']}
                            Content: {article['content']}"""
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            seed=21,
            temperature=0.5,
            messages=[
                {"role": "system", "content": gpt_role},
                {"role": "user", "content": gpt_prompt}
            ]
        )
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [17]:
calificaciones = []
for article in articles[0:3]:
    calificaciones.append(rank_article(article))

In [18]:
for i, calificacion in enumerate(calificaciones):
    print(f"{articles[i]['title']} -> {calificacion.choices[0].message.content}")

Tesla: Nuevo León anuncia licitación para la construcción de un puente de acceso a la planta -> { "total_score": 24 }
Ganancias de Sony caen 29% en el 2TF -> {"total_score": 21}
Santander y GAC Motor ofrecerán financiamiento de autos -> {
  "total_score": 25
}
Aceleran reactivación de Acapulco: hoteles prevén reabrir el 15 de diciembre -> {
   "total_score": 21
}
Softbank cae en el 2T fiscal tras golpe de WeWork -> {
  "total_score": 23
}


In [31]:
articulos_importancia_variada = []
articulos_importancia_variada.append(articles[37])
articulos_importancia_variada.append(articles[84]) # Articulo baja importancia 86
articulos_importancia_variada.append(articles[10])

In [34]:
print(rank_article(articulos_importancia_variada[0]).choices[0].message.content)

{
  "total_score": 17
}


In [46]:
calificaciones_test = []
for article in articulos_importancia_variada:
    calificaciones_test.append(rank_article(article))

In [47]:
for i, calificacion in enumerate(calificaciones_test):
    print(f"{articulos_importancia_variada[i]['title']} -> {calificacion.choices[0].message.content}")

Wozniak, cofundador de Apple, deja hospital en CDMX tras sufrir derrame cerebral menor -> {
  "total_score": 25
}
Virgin Galactic sube en Bolsa tras anunciar pausa a vuelos -> {
    "total_score": 40
}
Wall Street cierra negativo -> {
  "total_score": 28
}
